In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
from unet_prob import *

Using TensorFlow backend.


In [3]:
unet_prob = Unet_Prob()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 512, 512, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 512, 512, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 512, 512, 64) 36928       batch_normalization_1[0][0]      
__________________________________________________________________________________________________
batch_norm

In [4]:
#get data
TRAIN_PATH_IN = '/media/oala/4TB/DATA/experiments-hhi/uncertainty-quant/ellipsoid-toy/denoising/train/in/'
VAL_PATH_IN = '/media/oala/4TB/DATA/experiments-hhi/uncertainty-quant/ellipsoid-toy/denoising/val/in/'
TEST_PATH_IN = '/media/oala/4TB/DATA/experiments-hhi/uncertainty-quant/ellipsoid-toy/denoising/test/in/'

TRAIN_PATH_REF = '/media/oala/4TB/DATA/experiments-hhi/uncertainty-quant/ellipsoid-toy/denoising/train/ref/'
VAL_PATH_REF = '/media/oala/4TB/DATA/experiments-hhi/uncertainty-quant/ellipsoid-toy/denoising/val/ref/'
TEST_PATH_REF = '/media/oala/4TB/DATA/experiments-hhi/uncertainty-quant/ellipsoid-toy/denoising/test/ref/'


def get_data(path):
    TRAIN_PATH = path
    train_names = next(os.walk(TRAIN_PATH))[2]
    X_train = np.zeros((len(train_names), 512, 512), dtype=np.float64)
    print('Getting and resizing train images and masks ... ')
    for image_name, count in zip(train_names, range(len(train_names))):
        img = imread(TRAIN_PATH + image_name)
        #img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
        X_train[count] = img
        if count % 500 == 0:
            print('Done with # ', count)
    X_train = X_train[:,:,:,np.newaxis]
    
    return X_train

X_train = get_data(TRAIN_PATH_IN)
Y_train = get_data(TRAIN_PATH_REF)

X_val = get_data(VAL_PATH_IN)
Y_val = get_data(VAL_PATH_REF)

Getting and resizing train images and masks ... 
Done with #  0
Done with #  500
Done with #  1000
Done with #  1500
Getting and resizing train images and masks ... 
Done with #  0
Done with #  500
Done with #  1000
Done with #  1500
Getting and resizing train images and masks ... 
Done with #  0
Getting and resizing train images and masks ... 
Done with #  0


In [5]:
print(X_train.shape)

(1600, 512, 512, 1)


In [6]:
#scale 0 to 1
X_train = X_train / 65535.
Y_train = Y_train / 65535.
X_val = X_val / 65535.
Y_val = Y_val / 65535.

In [7]:
print(np.amax(X_train), np.amin(X_train), np.amax(Y_train), np.amin(Y_train))
print(np.amax(X_val), np.amin(X_val), np.amax(Y_val), np.amin(Y_val))

1.0 0.0 1.0 0.0
1.0 0.0 1.0 0.0


In [8]:
#get mean
#mean_X_train = np.mean(X_train, axis=0)
#mean_Y_train = np.mean(Y_train, axis=0)

#print(mean_X_train.shape, mean_Y_train.shape)

In [9]:
#demean train data
#X_train -= mean_X_train
#Y_train -= mean_X_train

In [10]:
#get std of demeaned train data
#std_X_train = np.std(X_train, axis = 0)
#std_Y_train = np.std(X_train, axis = 0)

In [11]:
#normalize train with std
#X_train /= std_X_train
#Y_train /= std_Y_train

In [12]:
#demean val, normalize val with std
#X_val = (X_val-mean_X_train)/std_X_train
#Y_val = (Y_val-mean_Y_train)/std_Y_train

In [13]:
#demean val
#X_val = X_val-mean_X_train
#Y_val = Y_val-mean_X_train

In [14]:
#print(np.amax(X_train), np.amin(X_train), np.amax(Y_train), np.amin(Y_train))
#print(np.amax(X_val), np.amin(X_val), np.amax(Y_val), np.amin(Y_val))

In [15]:
unet_prob.train(30, 1, X_train, Y_train, X_val, Y_val, X_val, Y_val)

Epoch 0 of 30
Train loss: 3508283.4867578126
Val loss: 289104.4865625
Took 807.9411318302155 seconds


Epoch 1 of 30
Train loss: 256891.54461914062
Val loss: 219965.76984375
Took 805.8215618133545 seconds


Epoch 2 of 30
Train loss: 165187.5170361328
Val loss: 107782.96451171875
Took 805.285566329956 seconds


Epoch 3 of 30
Train loss: 45657.42359375
Val loss: 2389.603837890625
Took 805.2735459804535 seconds


Epoch 4 of 30
Train loss: -35883.429322509764
Val loss: -65638.983125
Took 805.0143568515778 seconds


Epoch 5 of 30
Train loss: -103105.44914794921
Val loss: -139333.5352734375
Took 804.9022388458252 seconds


Epoch 6 of 30
Train loss: -178939.6884326172
Val loss: -220415.64390625
Took 805.6377758979797 seconds


Epoch 7 of 30
Train loss: -265752.34606445313
Val loss: -311390.501171875
Took 805.502622127533 seconds


Epoch 8 of 30
Train loss: -375858.32390625
Val loss: -439621.501875
Took 804.7440009117126 seconds


Epoch 9 of 30
Train loss: -520675.2919140625
Val loss: -606045.

KeyboardInterrupt: 

In [18]:
#unet_prob.save('unet_prob_linearprevar-linearmean-onedemean4all.h5')

In [16]:
#unet_prob.unet.load_weights('unet_prob_linearmean_linearvar.h5', by_name=True)

In [17]:
#a = unet_prob.unet.get_weights(conv1)